In [186]:
import numpy as np
import pandas as pd
import plotly.express as px

In [187]:
class ODE:
    def __init__(self, N):
        self.state = np.zeros(N, dtype=float)
        self.rate = np.zeros(N, dtype=float)

    def getState(self):
        return np.copy(self.state)
    
    def getRate(self):
        return np.copy(self.rate)
    
class ODESolver:
    def __init__(self, stepSize=0.01):
        self.dt = stepSize
    
    def step(self):
        pass

    def setStepSize(self, stepSize):
        self.dt = stepSize

    def getStepSize(self):
        return self.dt
    
class Euler(ODESolver):
    def step(self, other):
        state = np.array(other.getState())
        rate = np.array(other.getRate())
        other.state = state + rate*self.getStepSize()

class EulerRichardson(ODESolver):
    def step(self, other):
        state = np.array(other.getState())
        rate = np.array(other.getRate())

        other.state = state + (1/2)*rate*self.getStepSize()
        rate = other.getRate()

        other.state = state + rate*self.getStepSize()

class RungeKutta(ODESolver):
    def step(self, other):
        state0 = np.array(other.getState())
        rate0 = np.array(other.getRate())

        other.state = state0 + (1/2)*rate0*self.getStepSize()
        rate1 = np.array(other.getRate())

        other.state = state0 + (1/2)*rate1*self.getStepSize()
        rate2 = np.array(other.getRate())

        other.state = state0 + rate2*self.getStepSize()
        rate3 = np.array(other.getRate())

        other.state = state0 + (1/6)*(rate0 + 2*rate1 + 2*rate2 + rate3)*self.getStepSize()

In [188]:
class ProjectileODE(ODE):
    gx = 0
    gy = -9.81
    def __init__(self, x0, x0Dot, y0, y0Dot, t0=0):
        super().__init__(N=5)
        self.state[0] = x0
        self.state[1] = x0Dot
        self.state[2] = y0
        self.state[3] = y0Dot
        self.state[4] = t0

        self.xList = [x0]
        self.xDotList = [x0Dot]
        self.yList = [y0]
        self.yDotList = [y0Dot]
        self.tList = [t0]
        self.HistoryLists = [self.xList, self.xDotList, self.yList, self.yDotList, self.tList]

        self.x0 = x0
        self.y0 = y0
        self.vx0 = x0Dot
        self.vy0 = y0Dot
        self.t0 = t0

    def getRate(self):
        self.rate[0] = self.state[1]
        self.rate[1] = self.gx
        self.rate[2] = self.state[3]
        self.rate[3] = self.gy
        self.rate[4] = 1
        return self.rate
    
    def solveODE(self, T):
        solver = RungeKutta()
        while self.state[-1] < T:
            solver.step(self)
            for n in range(5):
                self.HistoryLists[n].append(self.state[n])

    def plotSolution(self):
        df = pd.DataFrame()
        df['x'] = self.HistoryLists[0]
        df['y'] = self.HistoryLists[2]
        df['t'] = self.HistoryLists[-1]

        fig = px.scatter(df, x='x', y='y', color='t', title="Trajectory")
        fig.show()

    def getAnalyticalResults(self):
        x = self.x0 + (self.vx0)*(self.state[-1] - self.t0) + (1/2)*(self.gx)*(self.state[-1] - self.t0)**2
        print("Analytical x:", x)
        print("Numerical x:", self.state[0])
        print("Error:", np.abs(x - self.state[0]))
        print('')

        y = self.y0 + (self.vy0)*(self.state[-1] - self.t0) + (1/2)*(self.gy)*(self.state[-1] - self.t0)**2
        print("Analytical y:", y)
        print("Numerical y:", self.state[2])
        print("Error:", np.abs(y - self.state[2]))
        print('')

In [189]:
p1 = ProjectileODE(0, 100, 0, 200)
p1.solveODE(T=20)
p1.plotSolution()
p1.getAnalyticalResults()

Analytical x: 2000.0000000000327
Numerical x: 2000.0
Error: 3.2741809263825417e-11

Analytical y: 2038.0000000000014
Numerical y: 2038.0000000001253
Error: 1.2391865311656147e-10

